<a href="https://colab.research.google.com/github/sohrab-namazi/Hopfield-network-MNIST-Recognition/blob/main/CI992_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hopfield nwtwork

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Hopfield network structure

class HopfieldNetwork:
  def __init__(self, neuron_count, patterns_count, patterns):
    self.neuron_count = neuron_count
    self.patterns_count = patterns_count
    self.patterns = patterns
    self.weights = np.zeros((self.neuron_count, self.neuron_count), dtype=float)

  def sign(self, x):
    if x >= 0:
      return 1
    else:
      return -1
  
  def compute_weights(self):
      for i in range(self.patterns_count):
        pattern = self.patterns[i]
        for j in range(self.neuron_count):
          for k in range(self.neuron_count):
            if j != k:
              self.weights[j, k] += pattern[j] * pattern[k]
      print(self.weights)
      return self.weights
  
  
  def run(self, state):
    is_learned_pattern = False
    current_state = state.copy()
    while not is_learned_pattern:
      new_pattern = []
      #print("Current State: " + str(current_state))
      for i in range(self.neuron_count):
        summation = 0.0
        for j in range(self.neuron_count):
          if i != j:
            summation += current_state[j] * self.weights[i, j]
        output = self.sign(summation)
        new_pattern.append(output)
        current_state[i] = output
      if new_pattern == current_state:
        is_learned_pattern =  True
      else:
        current_state = new_pattern.copy()
    return current_state
    
   

In [3]:
# prove that pattern = (1, 1, 1, -1, -1, -1) is stable

neuron_count = 6
patterns_count = 2
pattern_1 = [1, 1, 1, -1, -1, -1]
pattern_2 = [1, 1, 1, -1, -1, -1]
state = [1, 1, 1, -1, -1, -1]
patterns = [pattern_1, pattern_2]

network = HopfieldNetwork(neuron_count, patterns_count, patterns)
weights = network.compute_weights()
print("Weights:\n" + str(weights))
result = network.run(state)
print("Resulting pattern: " + str(result))
print("So it directly was converted to itself in the first iteration, we can conclude that it is a STABLE state")

[[ 0.  2.  2. -2. -2. -2.]
 [ 2.  0.  2. -2. -2. -2.]
 [ 2.  2.  0. -2. -2. -2.]
 [-2. -2. -2.  0.  2.  2.]
 [-2. -2. -2.  2.  0.  2.]
 [-2. -2. -2.  2.  2.  0.]]
Weights:
[[ 0.  2.  2. -2. -2. -2.]
 [ 2.  0.  2. -2. -2. -2.]
 [ 2.  2.  0. -2. -2. -2.]
 [-2. -2. -2.  0.  2.  2.]
 [-2. -2. -2.  2.  0.  2.]
 [-2. -2. -2.  2.  2.  0.]]
Resulting pattern: [1, 1, 1, -1, -1, -1]
So it directly was converted to itself in the first iteration, we can conclude that it is a STABLE state


In [4]:
# Entering pattern (-1, 1, 1, -1, -1, -1) to the network

state = [-1, 1, 1, -1, -1, -1]
print("Weights:\n" + str(weights))
result = network.run(state)
print("Resulting pattern: " + str(result))
print("So it matches the first pattern which was [1, 1, 1, -1, -1, -1]")

Weights:
[[ 0.  2.  2. -2. -2. -2.]
 [ 2.  0.  2. -2. -2. -2.]
 [ 2.  2.  0. -2. -2. -2.]
 [-2. -2. -2.  0.  2.  2.]
 [-2. -2. -2.  2.  0.  2.]
 [-2. -2. -2.  2.  2.  0.]]
Resulting pattern: [1, 1, 1, -1, -1, -1]
So it matches the first pattern which was [1, 1, 1, -1, -1, -1]


## MNIST by Hopfield

In [ ]:
from PIL import Image, ImageFont
import cv2
import numpy as np
import random
import math

In [ ]:
def build_data(font_size):
  font = ImageFont.truetype("Arial.ttf", font_size)
  for char in "ABCDEFGHIJ":
    im = Image.Image()._new(font.getmask(char))
    im = im.resize((img_width, img_height), Image.ANTIALIAS)
    im.save(f"{char}_{font_size}.bmp")  

def convert_img_to_array(path):
  img = cv2.imread(path)
  gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return gray_img

def convert_2darray_to_1darray(arr):
  return arr.flatten()

def convert_array_to_binarr(arr):
  bin_arr = np.where(arr >= limit_zero_one, 1, -1)
  return bin_arr

def convert_image_to_binarray(path):
  two_d_arr = convert_img_to_array(path)
  one_d_arr = convert_2darray_to_1darray(two_d_arr)
  bin_arr = convert_array_to_binarr(one_d_arr)
  return bin_arr

def convert_images_to_binarrays():  
  bin_arrays = []
  for char in alphabets:
    path = char + "_" + str(font_size) + ".bmp"
    bin_arr = convert_image_to_binarray(path) 
    bin_arrays.append(bin_arr)
  return bin_arrays

def add_noise_to_bin_array(bin_array, noisy_cells_count):
  noisy_bin_array = bin_array.copy()
  noisy_cells = random.sample(range(0, img_size), noisy_cells_count)
  #print("noisy cells: " + str(noisy_cells))
  for noisy_cell in noisy_cells:
    current_value = bin_array[noisy_cell]
    noisy_value = None
    if current_value == 1:
      noisy_value = -1
    else:
      noisy_value = 1
    noisy_bin_array[noisy_cell] = noisy_value
  return noisy_bin_array
  
def compare_lists(list1, list2):
  for i in range(img_size):
    a = list1[i]
    b = list2[i]
    if a == b:
      continue
    else:
      print("Different")
      return
  print("Same")

def compute_patterns_diff_count(list1, list2):
  diff_count = 0
  for i in range(img_size):
    a = list1[i]
    b = list2[i]
    if a == b:
      continue
    else:
      diff_count += 1
  return diff_count
        
def compare_patterns(patterns):
  for i in range(patterns_count):
    for j in range(patterns_count):
      if i != j:
        compare_lists(patterns[i], patterns[j])


def find_closest_matching_pattern(input_pattern, patterns):
  i = 0
  result = None
  min_diff_count = math.inf
  for pattern in patterns:
    diff_count = compute_patterns_diff_count(pattern.tolist(), input_pattern)
    if diff_count < min_diff_count:
      min_diff_count = diff_count
      result = i
    i += 1
  return result

def run_test(test_count, noise_percentage):
  patterns = convert_images_to_binarrays()
  patterns_count = len(patterns)
  network = HopfieldNetwork(img_width * img_height, patterns_count, patterns)
  network.compute_weights()
  true_diagnosis_count = 0
  for i in range(test_count):
    pattern_index = random.randint(0, patterns_count - 1)
    #print("noisy pattern index: " + str(pattern_index))
    pattern = patterns[pattern_index]
    noisy_cells_count = int(noise_percentage * img_size / 100)
    input = add_noise_to_bin_array(pattern, noisy_cells_count)
    output = network.run(input.tolist())
    #print("Input: " + str(input.tolist()))
    #print("Output pattern: " + str(output))
    result_index = find_closest_matching_pattern(output, patterns)
    #print("resulting pattern index: " + str(result_index))
    if pattern_index == result_index:
      true_diagnosis_count += 1
      print("network successfully verified character: " + alphabets[pattern_index]) 
    #print("\n")
  acc_percentage = (true_diagnosis_count / test_count * 100)
  print("******************************************\n\nTest finished, here is the result:\n\tAccruacy = " + str(acc_percentage) + "%")

In [ ]:
# set parameters (constant for all cases)
img_width = 10
img_height = 12
img_size = img_height * img_width
test_count = 10
limit_zero_one = 20
alphabets = "ABCDEFGHIJ"

In [ ]:
# noise = 60%, Font size = 16
noise_amount = 60 # noise amount unit is percantage
font_size = 16
build_data(font_size)
run_test(test_count, noise_amount)

[[ 0.  4.  0. ... -6.  0.  6.]
 [ 4.  0.  6. ... -4. -2.  0.]
 [ 0.  6.  0. ...  0. -2. -4.]
 ...
 [-6. -4.  0. ...  0.  4. -6.]
 [ 0. -2. -2. ...  4.  0.  0.]
 [ 6.  0. -4. ... -6.  0.  0.]]
network successfully verified character: A
******************************************

Test finished, here is the result:
	Accruacy = 10.0%


In [ ]:
# noise = 30%, Font size = 16
noise_amount = 30 # noise amount unit is percantage
font_size = 16
build_data(font_size)
run_test(test_count, noise_amount)

[[ 0.  4.  0. ... -6.  0.  6.]
 [ 4.  0.  6. ... -4. -2.  0.]
 [ 0.  6.  0. ...  0. -2. -4.]
 ...
 [-6. -4.  0. ...  0.  4. -6.]
 [ 0. -2. -2. ...  4.  0.  0.]
 [ 6.  0. -4. ... -6.  0.  0.]]
network successfully verified character: C
network successfully verified character: C
network successfully verified character: C
network successfully verified character: H
network successfully verified character: A
network successfully verified character: B
******************************************

Test finished, here is the result:
	Accruacy = 60.0%


In [ ]:
# noise = 10%, Font size = 16
noise_amount = 10 # noise amount unit is percantage
font_size = 16
build_data(font_size)
run_test(test_count, noise_amount)

[[ 0.  4.  0. ... -6.  0.  6.]
 [ 4.  0.  6. ... -4. -2.  0.]
 [ 0.  6.  0. ...  0. -2. -4.]
 ...
 [-6. -4.  0. ...  0.  4. -6.]
 [ 0. -2. -2. ...  4.  0.  0.]
 [ 6.  0. -4. ... -6.  0.  0.]]
network successfully verified character: C
network successfully verified character: I
network successfully verified character: C
network successfully verified character: C
network successfully verified character: A
network successfully verified character: C
network successfully verified character: B
******************************************

Test finished, here is the result:
	Accruacy = 70.0%


In [ ]:
# noise = 60%, Font size = 32
test_count = 100
noise_amount = 60 # noise amount unit is percantage
font_size = 32
build_data(font_size)
run_test(test_count, noise_amount)

[[ 0.  0. -4. ... -6.  2.  6.]
 [ 0.  0.  6. ...  0.  4.  0.]
 [-4.  6.  0. ...  4.  0. -4.]
 ...
 [-6.  0.  4. ...  0.  2. -2.]
 [ 2.  4.  0. ...  2.  0.  6.]
 [ 6.  0. -4. ... -2.  6.  0.]]
network successfully verified character: I
network successfully verified character: I
network successfully verified character: I
******************************************

Test finished, here is the result:
	Accruacy = 3.0%


In [ ]:
# noise = 30%, Font size = 32
noise_amount = 30 # noise amount unit is percantage
font_size = 32
build_data(font_size)
run_test(test_count, noise_amount)

[[ 0.  0. -4. ... -6.  2.  6.]
 [ 0.  0.  6. ...  0.  4.  0.]
 [-4.  6.  0. ...  4.  0. -4.]
 ...
 [-6.  0.  4. ...  0.  2. -2.]
 [ 2.  4.  0. ...  2.  0.  6.]
 [ 6.  0. -4. ... -2.  6.  0.]]
network successfully verified character: I
network successfully verified character: I
network successfully verified character: D
network successfully verified character: D
network successfully verified character: I
network successfully verified character: B
network successfully verified character: I
network successfully verified character: B
network successfully verified character: A
network successfully verified character: H
network successfully verified character: B
network successfully verified character: D
network successfully verified character: D
network successfully verified character: J
network successfully verified character: D
network successfully verified character: D
network successfully verified character: I
network successfully verified character: A
network successfully verified char

In [ ]:
# noise = 10%, Font size = 32
noise_amount = 10 # noise amount unit is percantage
font_size = 32
build_data(font_size)
run_test(test_count, noise_amount)

[[ 0.  0. -4. ... -6.  2.  6.]
 [ 0.  0.  6. ...  0.  4.  0.]
 [-4.  6.  0. ...  4.  0. -4.]
 ...
 [-6.  0.  4. ...  0.  2. -2.]
 [ 2.  4.  0. ...  2.  0.  6.]
 [ 6.  0. -4. ... -2.  6.  0.]]
network successfully verified character: A
network successfully verified character: B
network successfully verified character: A
network successfully verified character: I
network successfully verified character: I
network successfully verified character: A
network successfully verified character: A
network successfully verified character: A
network successfully verified character: I
network successfully verified character: D
network successfully verified character: B
network successfully verified character: G
network successfully verified character: D
network successfully verified character: A
network successfully verified character: B
network successfully verified character: A
network successfully verified character: D
network successfully verified character: A
network successfully verified char

In [ ]:
# noise = 60%, Font size = 64
noise_amount = 60 # noise amount unit is percantage
font_size = 64
build_data(font_size)
run_test(test_count, noise_amount)

[[ 0.  0. -2. ... -4.  4.  6.]
 [ 0.  0.  8. ...  2.  2.  0.]
 [-2.  8.  0. ...  4.  0. -2.]
 ...
 [-4.  2.  4. ...  0. -2. -4.]
 [ 4.  2.  0. ... -2.  0.  8.]
 [ 6.  0. -2. ... -4.  8.  0.]]
network successfully verified character: I
network successfully verified character: I
******************************************

Test finished, here is the result:
	Accruacy = 2.0%


In [ ]:
# noise = 30%, Font size = 64
noise_amount = 30 # noise amount unit is percantage
font_size = 64
build_data(font_size)
run_test(test_count, noise_amount)

[[ 0.  0. -2. ... -4.  4.  6.]
 [ 0.  0.  8. ...  2.  2.  0.]
 [-2.  8.  0. ...  4.  0. -2.]
 ...
 [-4.  2.  4. ...  0. -2. -4.]
 [ 4.  2.  0. ... -2.  0.  8.]
 [ 6.  0. -2. ... -4.  8.  0.]]
network successfully verified character: A
network successfully verified character: J
network successfully verified character: B
network successfully verified character: I
network successfully verified character: B
network successfully verified character: D
network successfully verified character: D
network successfully verified character: B
network successfully verified character: D
network successfully verified character: D
network successfully verified character: D
network successfully verified character: D
network successfully verified character: B
network successfully verified character: B
network successfully verified character: B
network successfully verified character: I
network successfully verified character: I
network successfully verified character: B
network successfully verified char

In [ ]:
# noise = 60%, Font size = 64
noise_amount = 10 # noise amount unit is percantage
font_size = 64
build_data(font_size)
run_test(test_count, noise_amount)

[[ 0.  0. -2. ... -4.  4.  6.]
 [ 0.  0.  8. ...  2.  2.  0.]
 [-2.  8.  0. ...  4.  0. -2.]
 ...
 [-4.  2.  4. ...  0. -2. -4.]
 [ 4.  2.  0. ... -2.  0.  8.]
 [ 6.  0. -2. ... -4.  8.  0.]]
network successfully verified character: B
network successfully verified character: B
network successfully verified character: B
network successfully verified character: D
network successfully verified character: A
network successfully verified character: B
network successfully verified character: I
network successfully verified character: B
network successfully verified character: A
network successfully verified character: A
network successfully verified character: I
network successfully verified character: A
network successfully verified character: A
network successfully verified character: A
network successfully verified character: I
network successfully verified character: A
network successfully verified character: A
network successfully verified character: J
network successfully verified char